# Factory Sensor Data Analysis and Reporting

In [246]:
import pandas as pd

## Dataset

In [247]:
df = pd.read_csv('Data/industrial_sensor_data.csv', parse_dates=['timestamp'])

In [248]:
pd.options.display.max_rows = 10

In [249]:
df['timestamp'] = df['timestamp'].dt.ceil('S')

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\1873756507.py:1: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['timestamp'] = df['timestamp'].dt.ceil('S')


In [250]:
df.head()

,timestamp,machine_id,sensor_type,sensor_value,unit
0,2025-09-27 02:28:45,M07,pressure,3.26,bar
1,2025-09-26 03:13:45,M02,temperature,80.96,C
2,2025-09-19 07:18:45,M09,pressure,4.81,bar
3,2025-09-13 13:18:45,M01,pressure,3.68,bar
4,2025-09-28 20:23:45,M06,temperature,79.97,C


In [251]:
len(df)

259200

## Data Cleaning

In [252]:
df.isnull().sum()

timestamp       0
machine_id      0
sensor_type     0
sensor_value    0
unit            0
dtype: int64

In [253]:
df.duplicated().value_counts()

False    259200
Name: count, dtype: int64

In [254]:
df2 = df.copy()

In [255]:
df2['unit'][df2['unit']=='C'] = 'Celsius'

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\1812997798.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df2['unit'][df2['unit']=='C'] = 'Celsius'
C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\1812997798.py:1: SettingWith

In [256]:
df2

,timestamp,machine_id,sensor_type,sensor_value,unit
0,2025-09-27 02:28:45,M07,pressure,3.260,bar
1,2025-09-26 03:13:45,M02,temperature,80.960,Celsius
2,2025-09-19 07:18:45,M09,pressure,4.810,bar
3,2025-09-13 13:18:45,M01,pressure,3.680,bar
4,2025-09-28 20:23:45,M06,temperature,79.970,Celsius
...,...,...,...,...,...
259195,2025-09-14 14:18:45,M07,temperature,71.590,Celsius
259196,2025-09-14 14:23:45,M09,pressure,4.330,bar
259197,2025-09-28 13:18:45,M03,pressure,3.380,bar
259198,2025-09-22 10:53:45,M07,vibration,0.079,mm/s


## Data Transformation

### Pivot the table to have each sensor as a column

In [257]:
df_pivot = df.pivot(index=['timestamp','machine_id'], columns='sensor_type', values='sensor_value').reset_index().rename_axis(None, axis=1)

In [258]:
df_pivot

,timestamp,machine_id,pressure,temperature,vibration
0,2025-08-30 15:43:45,M01,3.91,83.38,0.089
1,2025-08-30 15:43:45,M02,4.12,70.24,0.023
2,2025-08-30 15:43:45,M03,4.29,84.94,0.088
3,2025-08-30 15:43:45,M04,2.20,74.16,0.019
4,2025-08-30 15:43:45,M05,2.21,86.23,0.050
...,...,...,...,...,...
86395,2025-09-29 15:38:45,M06,3.96,73.35,0.049
86396,2025-09-29 15:38:45,M07,4.04,69.75,0.073
86397,2025-09-29 15:38:45,M08,3.45,84.86,0.044
86398,2025-09-29 15:38:45,M09,1.63,81.68,0.097


### Resample data to hourly/daily averages

In [259]:
df3 = df.set_index('timestamp')

In [260]:
df3

,machine_id,sensor_type,sensor_value,unit
timestamp,,,,
2025-09-27 02:28:45,M07,pressure,3.260,bar
2025-09-26 03:13:45,M02,temperature,80.960,C
2025-09-19 07:18:45,M09,pressure,4.810,bar
2025-09-13 13:18:45,M01,pressure,3.680,bar
2025-09-28 20:23:45,M06,temperature,79.970,C
...,...,...,...,...
2025-09-14 14:18:45,M07,temperature,71.590,C
2025-09-14 14:23:45,M09,pressure,4.330,bar
2025-09-28 13:18:45,M03,pressure,3.380,bar


In [261]:
df3.resample('h').sum(numeric_only=True)

,sensor_value
timestamp,
2025-08-30 15:00:00,3290.109
2025-08-30 16:00:00,9565.451
2025-08-30 17:00:00,9742.809
2025-08-30 18:00:00,9757.657
2025-08-30 19:00:00,9626.439
...,...
2025-09-29 11:00:00,9708.624
2025-09-29 12:00:00,9666.141
2025-09-29 13:00:00,9782.272


In [262]:
df3.resample('D').sum(numeric_only=True)

,sensor_value
timestamp,
2025-08-30,80992.756
2025-08-31,232273.644
2025-09-01,232162.196
2025-09-02,231813.986
2025-09-03,231728.607
...,...
2025-09-25,231904.599
2025-09-26,232361.542
2025-09-27,232117.822


In [263]:
df4 = df.groupby(['machine_id','sensor_type',pd.Grouper(key='timestamp', freq='h')]).mean(numeric_only=True).reset_index()

In [264]:
df4

,machine_id,sensor_type,timestamp,sensor_value
0,M01,pressure,2025-08-30 15:00:00,3.025000
1,M01,pressure,2025-08-30 16:00:00,3.330000
2,M01,pressure,2025-08-30 17:00:00,3.068333
3,M01,pressure,2025-08-30 18:00:00,3.552500
4,M01,pressure,2025-08-30 19:00:00,2.893333
...,...,...,...,...
21625,M10,vibration,2025-09-29 11:00:00,0.052417
21626,M10,vibration,2025-09-29 12:00:00,0.061583
21627,M10,vibration,2025-09-29 13:00:00,0.042667
21628,M10,vibration,2025-09-29 14:00:00,0.049750


In [265]:
df5 = df.groupby(['machine_id','sensor_type',pd.Grouper(key='timestamp', freq='D')]).mean(numeric_only=True).reset_index()

In [266]:
df5

,machine_id,sensor_type,timestamp,sensor_value
0,M01,pressure,2025-08-30,3.045300
1,M01,pressure,2025-08-31,2.920556
2,M01,pressure,2025-09-01,3.003611
3,M01,pressure,2025-09-02,2.975903
4,M01,pressure,2025-09-03,2.978090
...,...,...,...,...
925,M10,vibration,2025-09-25,0.051802
926,M10,vibration,2025-09-26,0.053903
927,M10,vibration,2025-09-27,0.054844
928,M10,vibration,2025-09-28,0.053507


### Compute rolling statistics

In [267]:
df6 = df.set_index('timestamp')

In [268]:
df6.sort_index(inplace=True)

In [269]:
df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\1753753003.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])


,machine_id,sensor_type,sensor_value,unit,rolling_mean,rolling_std
timestamp,,,,,,
2025-08-30 15:43:45,M01,pressure,3.910,bar,3.910000,NaN
2025-08-30 15:48:45,M01,pressure,1.790,bar,2.850000,1.499066
2025-08-30 15:53:45,M01,pressure,1.800,bar,2.500000,1.221106
2025-08-30 15:58:45,M01,pressure,4.600,bar,3.025000,1.447953
2025-08-30 16:03:45,M01,pressure,3.870,bar,3.194000,1.309668
...,...,...,...,...,...,...
2025-09-29 15:18:45,M10,vibration,0.031,mm/s,0.048417,0.022625
2025-09-29 15:23:45,M10,vibration,0.043,mm/s,0.048417,0.022625
2025-09-29 15:28:45,M10,vibration,0.091,mm/s,0.051500,0.025759


In [270]:
df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1D').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\2746412287.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1D').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])


,machine_id,sensor_type,sensor_value,unit,rolling_mean,rolling_std
timestamp,,,,,,
2025-08-30 15:43:45,M01,pressure,3.910,bar,3.910000,NaN
2025-08-30 15:48:45,M01,pressure,1.790,bar,2.850000,1.499066
2025-08-30 15:53:45,M01,pressure,1.800,bar,2.500000,1.221106
2025-08-30 15:58:45,M01,pressure,4.600,bar,3.025000,1.447953
2025-08-30 16:03:45,M01,pressure,3.870,bar,3.194000,1.309668
...,...,...,...,...,...,...
2025-09-29 15:18:45,M10,vibration,0.031,mm/s,0.053476,0.022625
2025-09-29 15:23:45,M10,vibration,0.043,mm/s,0.053444,0.022625
2025-09-29 15:28:45,M10,vibration,0.091,mm/s,0.053562,0.025759


In [271]:
df[df['sensor_type']=='temperature'].max()

timestamp       2025-09-29 15:38:45
machine_id                      M10
sensor_type             temperature
sensor_value                   90.0
unit                              C
dtype: object

In [272]:
df[df['sensor_type']=='temperature'].min()

timestamp       2025-08-30 15:43:45
machine_id                      M01
sensor_type             temperature
sensor_value                   65.0
unit                              C
dtype: object

In [273]:
df7 = df[df['sensor_type']=='temperature']

In [274]:
temp_anomaly = df7[(df7['sensor_value']<70) | (df7['sensor_value']>85)]

In [275]:
temp_anomaly

,timestamp,machine_id,sensor_type,sensor_value,unit
21,2025-09-19 12:18:45,M03,temperature,85.60,C
30,2025-09-28 18:13:45,M09,temperature,67.04,C
42,2025-09-27 08:58:45,M09,temperature,69.13,C
43,2025-09-26 03:18:45,M07,temperature,86.41,C
48,2025-09-24 13:03:45,M10,temperature,85.50,C
...,...,...,...,...,...
259151,2025-09-18 11:08:45,M03,temperature,87.86,C
259154,2025-09-05 02:48:45,M07,temperature,66.15,C
259169,2025-09-24 07:08:45,M05,temperature,65.97,C
259171,2025-09-26 11:13:45,M09,temperature,65.49,C


In [276]:
temp_anomaly.sort_values(by=['machine_id','timestamp']).set_index('timestamp')

,machine_id,sensor_type,sensor_value,unit
timestamp,,,,
2025-08-30 16:23:45,M01,temperature,86.58,C
2025-08-30 16:28:45,M01,temperature,66.58,C
2025-08-30 16:48:45,M01,temperature,87.94,C
2025-08-30 16:58:45,M01,temperature,66.43,C
2025-08-30 17:18:45,M01,temperature,86.84,C
...,...,...,...,...
2025-09-29 14:43:45,M10,temperature,86.57,C
2025-09-29 15:03:45,M10,temperature,85.35,C
2025-09-29 15:13:45,M10,temperature,67.35,C


In [277]:
df8 = df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\3780566630.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df8 = df6.groupby(['machine_id','sensor_type'], group_keys=False).apply(lambda x: x.assign(rolling_mean = x['sensor_value'].rolling('1h').mean(), rolling_std = x['sensor_value'].rolling('1h').std())).sort_values(by=['machine_id','sensor_type'])


In [278]:
vib_anomaly = df8[(df8['sensor_value'] - df8['rolling_mean']).abs() > 2.5*df8['rolling_std']]

In [279]:
vib_anomaly

,machine_id,sensor_type,sensor_value,unit,rolling_mean,rolling_std
timestamp,,,,,,
2025-09-01 06:03:45,M01,pressure,4.260,bar,2.191667,0.784843
2025-09-26 18:03:45,M01,pressure,4.940,bar,2.745833,0.839388
2025-09-29 11:58:45,M01,pressure,1.290,bar,3.400833,0.842167
2025-09-11 14:58:45,M01,temperature,67.180,C,80.484167,4.934510
2025-09-18 22:18:45,M01,temperature,65.120,C,82.451667,6.765658
...,...,...,...,...,...,...
2025-09-19 18:48:45,M10,temperature,65.920,C,84.811667,7.051789
2025-09-05 18:28:45,M10,vibration,0.016,mm/s,0.070083,0.020839
2025-09-24 01:13:45,M10,vibration,0.087,mm/s,0.035917,0.020237


In [280]:
temp_anomaly['anomaly'] = 'temperature anomaly'

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\477448550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_anomaly['anomaly'] = 'temperature anomaly'


In [281]:
vib_anomaly['anomaly'] = 'vibration spike'

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\4031240392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vib_anomaly['anomaly'] = 'vibration spike'


In [282]:
anomaly_table = df.merge(temp_anomaly, how='outer')

In [283]:
anomaly_table

,timestamp,machine_id,sensor_type,sensor_value,unit,anomaly
0,2025-08-30 15:43:45,M01,pressure,3.910,bar,NaN
1,2025-08-30 15:43:45,M01,temperature,83.380,C,NaN
2,2025-08-30 15:43:45,M01,vibration,0.089,mm/s,NaN
3,2025-08-30 15:43:45,M02,pressure,4.120,bar,NaN
4,2025-08-30 15:43:45,M02,temperature,70.240,C,NaN
...,...,...,...,...,...,...
259195,2025-09-29 15:38:45,M09,temperature,81.680,C,NaN
259196,2025-09-29 15:38:45,M09,vibration,0.097,mm/s,NaN
259197,2025-09-29 15:38:45,M10,pressure,2.520,bar,NaN
259198,2025-09-29 15:38:45,M10,temperature,79.780,C,NaN


In [284]:
vib_anomaly.reset_index(inplace=True)

In [285]:
vib_anomaly.drop(columns=['rolling_mean','rolling_std'], inplace=True)

C:\Users\smazumdar\AppData\Local\Temp\ipykernel_11812\3819942728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vib_anomaly.drop(columns=['rolling_mean','rolling_std'], inplace=True)


In [286]:
anomaly_table = anomaly_table.merge(vib_anomaly, how='outer')

In [287]:
anomaly_table

,timestamp,machine_id,sensor_type,sensor_value,unit,anomaly
0,2025-08-30 15:43:45,M01,pressure,3.910,bar,NaN
1,2025-08-30 15:43:45,M01,temperature,83.380,C,NaN
2,2025-08-30 15:43:45,M01,vibration,0.089,mm/s,NaN
3,2025-08-30 15:43:45,M02,pressure,4.120,bar,NaN
4,2025-08-30 15:43:45,M02,temperature,70.240,C,NaN
...,...,...,...,...,...,...
259283,2025-09-29 15:38:45,M09,temperature,81.680,C,NaN
259284,2025-09-29 15:38:45,M09,vibration,0.097,mm/s,NaN
259285,2025-09-29 15:38:45,M10,pressure,2.520,bar,NaN
259286,2025-09-29 15:38:45,M10,temperature,79.780,C,NaN
